#  Homework 4 for CS 247 : Advanced Data Mining Learning


### Due: 11:59 pm 04/29

##### Please read the Homework Guidance (uploaded to CCLE) carefully and make sure you fulfill all the requirements.



__Name__: [Wei Zhou]

__UID__: [505650843]

## Problem 1: Multinomial Mixture Model (30 pts)

Similar to Gaussian Mixture Model, Multinomial Mixture Model can also be used for soft clustering. 
For example, in the case of document clustering, each cluster $z$ is a multinomial distribution with parameters $\beta_k$, and $x_i|z_i \sim mutlinomial(\beta_k)$, where $x_i$ is a bag-of-words representation vector for document $i$. Please write down the EM algorithm for soft document clustering under Multinomial Mixture Model.

#### Write Your answer here:

[Your Answer]
E step: https://drive.google.com/file/d/13_5tKOKJehiu-cNPm4OsNZ5RAebW5NIK/view?usp=sharing 

M step: https://drive.google.com/file/d/1Z6CeZFSVLG3muH6kNxgjGxtpFhGarxWe/view?usp=sharing



## Problem 2: Probabilistic Latent Semantic Analysis （70 pts)

In this problem, you are going to implement the pLSA algorithm on the dataset, and answer some questions about pLSA Initialization.


### Part 1: pLSA Implementation (45pts = 15 + 15 + 15)

In this part, you are going to implement the pLSA algorithm. We provide 2 datasets for you. Dataset1 is very simple and it's mainly for you to check whether your implemenation is correct or not. Dataset2 is more complicated. **Please report your results with dataset2**. Please note that, for dataset1, you should set num_topics=4, and for dataset2, you should set num_topics=10.

Hint: You can check your implemenation with dataset 1. If your implementation is correct, you should be able to get the following results (please do not change any of our parameter setting and random seeds):

['dragon', 'ball', 'kakarot', 'world', 'action']

['dragon', 'ball', 'kakarot', 'anim', 'seri']

['covid', 'lab', 'coronaviru', 'california', 'lo']

['covid', 'wuhan', 'person', 'spy', 'agenc']

In [5]:
from google.colab import drive

drive.mount('/gdrive')


Mounted at /gdrive


In [3]:
# import neccessary libraries

import numpy as np # please update your numpy version if you get the error "module 'numpy.random' has no attribute 'default_rng'"
from gensim.parsing.preprocessing import preprocess_string
from collections import defaultdict

In [6]:
# load and pre-process data
# word2id : a map mapping terms to their corresponding ids
# id2word : a map mapping ids to terms
# X : document-word matrix, N*M, each line is the number of terms that show up in the document


wordCounts = []
word2id = {}
id2word = {}

datasetFilePath = '/gdrive/MyDrive/Colab Notebooks/CS247/hw4/dataset2.txt' 
# you can use dataset1 to check your implementation, but you should use dataset2 to report your results#
num_topics = 10
# for dataset1, please use num_topics=4; for dataset2, please use num_topics=10#

fin = open(datasetFilePath,encoding='UTF-8')
documents = fin.readlines()
for doc in documents:
    words = preprocess_string(doc)
    word_count = defaultdict(lambda: 0)
    for word in words:
        if word not in word2id:
            idx = len(id2word)
            word2id[word] = idx
            id2word[idx]  = word
        word_count[word] += 1
    wordCounts += [word_count]
    
    
num_doc = len(documents)
num_words = len(word2id)  

# generate the document-word matrix
X = np.zeros([num_doc, num_words], np.int)

for word in word2id:
    j = word2id[word]
    for i in range(0, num_doc):
        if word in wordCounts[i]:
            X[i, j] = wordCounts[i][word]
            

print("Number of Documents: ", num_doc)
print("Number of Words in the Vocabulary: ", num_words)

Number of Documents:  16
Number of Words in the Vocabulary:  728


In [38]:
# pLSA class

class PLSA():
    def __init__(self, num_doc, num_words, num_topics):
        
        self.num_doc = num_doc
        self.num_words = num_words
        self.num_topics = num_topics
        
        # theta_dz: topic distribution for each document p(z|d)
        np.random.seed(0)
        self.theta = np.random.random([num_doc, num_topics])

        # beta_zw:  word distribution for each topic p(w|z) 
        np.random.seed(1)
        self.beta = np.random.random([num_topics, num_words])

        # p[i, j, k] : lower bound of p(zk|di,wj)
        self.p = np.zeros([num_doc, num_words, num_topics])

        self.theta /= np.sum(self.theta, axis=1).reshape(-1,1)
        self.beta  /= np.sum(self.beta, axis=1).reshape(-1,1)

    def EStep(self):
        '''
        TODO:
            update lower bound p
        '''
        for i in range(0, num_doc):
          for j in range(0, num_words):
              denominator = 0;
              for k in range(0, num_topics):
                  self.p[i, j, k] = self.beta[k, j] * self.theta[i, k];
                  denominator += self.p[i, j, k];
              if denominator == 0:
                  for k in range(0, num_topics):
                      self.p[i, j, k] = 0;
              else:
                  for k in range(0, num_topics):
                      self.p[i, j, k] /= (denominator + 0.0000000001) 
                        
    def MStep(self, X):
        '''
        TODO:
            update beta
        '''
        for k in range(0, num_topics):
          denominator = 0
          for j in range(0, num_words):
              self.beta[k, j] = 0
              for i in range(0, num_doc):
                  self.beta[k, j] += X[i, j] * self.p[i, j, k]
              denominator += self.beta[k, j]
          if denominator == 0:
              for j in range(0, num_words):
                  self.beta[k, j] = 1.0 / num_words
          else:
              for j in range(0, num_words):
                  self.beta[k, j] /= (denominator + 0.0000000001)   
        
      
        
        '''
        TODO:
            update theta
        '''
        
        for i in range(0, num_doc):
            for k in range(0, num_topics):
                self.theta[i, k] = 0
                denominator = 0
                for j in range(0, num_words):
                    self.theta[i, k] += X[i, j] * self.p[i, j, k]
                    denominator += X[i, j];
                if denominator == 0:
                    self.theta[i, k] = 1.0 / num_topics
                else:
                    self.theta[i, k] /= (denominator +0.0000000001 )
                    
    # calculate the log likelihood
    def LogLikelihood(self, X):
        loglikelihood = 0
        for i in range(0, self.num_doc):
            for j in range(0, self.num_words):
                tmp = 0
                for k in range(0, self.num_topics):
                    tmp += self.beta[k, j] * self.theta[i, k]
                if tmp > 0:
                    loglikelihood += X[i, j] * np.log(tmp)
        return loglikelihood
    
    

In [39]:
# test the PLSA class

plsa = PLSA(num_doc, num_words, num_topics)

# EM algorithm
for i in range(0, 30):
    plsa.EStep()
    plsa.MStep(X)
    log_likelihood = plsa.LogLikelihood(X)
    print("iteration %d, LogLikelihood: %.3f" % (i, log_likelihood))

iteration 0, LogLikelihood: -8593.509
iteration 1, LogLikelihood: -8370.475
iteration 2, LogLikelihood: -8087.982
iteration 3, LogLikelihood: -7767.412
iteration 4, LogLikelihood: -7425.230
iteration 5, LogLikelihood: -7132.968
iteration 6, LogLikelihood: -6938.684
iteration 7, LogLikelihood: -6819.566
iteration 8, LogLikelihood: -6737.579
iteration 9, LogLikelihood: -6672.181
iteration 10, LogLikelihood: -6616.028
iteration 11, LogLikelihood: -6569.560
iteration 12, LogLikelihood: -6529.477
iteration 13, LogLikelihood: -6490.083
iteration 14, LogLikelihood: -6458.853
iteration 15, LogLikelihood: -6445.527
iteration 16, LogLikelihood: -6438.536
iteration 17, LogLikelihood: -6434.878
iteration 18, LogLikelihood: -6432.785
iteration 19, LogLikelihood: -6430.464
iteration 20, LogLikelihood: -6428.539
iteration 21, LogLikelihood: -6427.353
iteration 22, LogLikelihood: -6426.471
iteration 23, LogLikelihood: -6425.848
iteration 24, LogLikelihood: -6425.541
iteration 25, LogLikelihood: -6425.

In [40]:
# print the top-5 frequent words in each topic

for i in plsa.beta:
    topic_word = []
    for idx in (-i).argsort()[:5]:
        topic_word += [id2word[idx]]
    print(topic_word)

['luffi', 'dressrosa', 'pirat', 'ac', 'flame']
['zou', 'pirat', 'luffi', 'big', 'mom']
['haki', 'color', 'allow', 'snail', 'den']
['luffi', 'pirat', 'captain', 'crew', 'name']
['crew', 'pirat', 'govern', 'franki', 'hat']
['island', 'pose', 'field', 'log', 'magnet']
['fruit', 'devil', 'user', 'power', 'sea']
['manga', 'seri', 'anim', 'licens', 'piec']
['line', 'sea', 'grand', 'world', 'blue']
['pirat', 'baroqu', 'alabasta', 'work', 'line']


### Part 2: pLSA Initialization (25 pts = 10 + 15)

Please answer the following two questions about pLSA initialization:

1. Let K denote the number of topics, N denote the total number of words in the dictionary. Is it a good initialization to set θd’s and βk’s as uniform distribution, i.e., θdk = 1/K for every d and k, and  βkw = 1/N for every k and w, where N is the total number of words in the dictionary? Why (please give some formulas to illustrate your conclusion)? 

2. Can you give an example of bad initialization? (if the initialization in question 1 is a bad initialization, you **CANNOT** use it as answer for question 2) Why this is a bad initialization (please give some formulas to illustrate your conclusion)?


#### Write Your answer here:

https://drive.google.com/file/d/1V7D60ZxYxhuFrOty7d3Df4BEukeVYrnN/view?usp=sharing


# Citation

PLSA:

https://github.com/laserwave 
https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python
https://towardsdatascience.com/topic-modelling-with-plsa-728b92043f41
http://zhikaizhang.cn/2016/06/17/%E8%87%AA%E7%84%B6%E8%AF%AD%E8%A8%80%E5%A4%84%E7%90%86%E4%B9%8BPLSA/


Collaboration & help: Study group: Jessica Bojorquez, John Fritsche Lucius Wang, Office Hour content (Thank you Yewen! ) 


Initialization: 
https://towardsdatascience.com/random-initialization-for-neural-networks-a-thing-of-the-past-bfcdd806bf9e

